In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import optimize, OptimizeConfig, NSGAIIConfig
from vamos.foundation.problem.real_world.engineering import WeldedBeamDesignProblem
from vamos.foundation.problem.cec2009 import CEC2009_CF1
from vamos.foundation.constraints import (
    compute_constraint_info,
    FeasibilityFirstStrategy,
    PenaltyCVStrategy,
    EpsilonConstraintStrategy,
)

plt.style.use("ggplot")
print("Constraint handling modules loaded!")

## 1. Constraint Convention

In VAMOS (and most EMO frameworks), constraints follow the **"≤ 0 is feasible"** convention:

- `G[i, j] ≤ 0` → constraint j satisfied for solution i
- `G[i, j] > 0` → constraint j violated (violation amount = G[i, j])

Problems store constraints in `out["G"]` with shape `(n_points, n_constraints)`.

In [ ]:
# Example: manual constraint evaluation
G_example = np.array([
    [-0.5, -0.2],   # Both constraints satisfied (≤ 0)
    [0.3, -0.1],    # Constraint 1 violated by 0.3
    [0.0, 0.5],     # Constraint 2 violated by 0.5
    [0.2, 0.3],     # Both constraints violated
])

info = compute_constraint_info(G_example)

print("Constraint matrix G (≤ 0 = satisfied):")
print(G_example)
print()
print(f"Constraint violation (CV) per solution: {info.cv}")
print(f"Feasible mask: {info.feasible_mask}")
print(f"Number of feasible solutions: {info.feasible_mask.sum()} / {len(info.feasible_mask)}")

## 2. Constraint Handling Strategies

VAMOS supports multiple strategies for handling constraints during selection:

| Strategy | Description |
|----------|-------------|
| `FeasibilityFirstStrategy` | Feasible always beats infeasible; among infeasible, lower CV wins |
| `PenaltyCVStrategy` | Adds λ × CV to aggregated objectives |
| `EpsilonConstraintStrategy` | Feasibility-first with epsilon tolerance |
| `CVAsObjectiveStrategy` | Treats CV as a ranking criterion with objective tie-break |

In [ ]:
# Example: Compare strategies
F = np.array([
    [1.0, 2.0],   # Good objectives, feasible
    [0.5, 1.5],   # Better objectives, infeasible
    [1.2, 2.5],   # Worse objectives, feasible
    [0.8, 1.8],   # Medium objectives, slightly infeasible
])

G = np.array([
    [-0.1],       # Feasible
    [0.5],        # Infeasible (CV = 0.5)
    [-0.2],       # Feasible
    [0.1],        # Slightly infeasible (CV = 0.1)
])

strategies = {
    "FeasibilityFirst": FeasibilityFirstStrategy("sum"),
    "Penalty (λ=10)": PenaltyCVStrategy(penalty_lambda=10.0),
    "Epsilon (ε=0.15)": EpsilonConstraintStrategy(epsilon=0.15),
}

print("Solution ranks (lower = better):")
print("-" * 60)
for name, strategy in strategies.items():
    ranks = strategy.rank(F, G)
    print(f"{name:20s}: {ranks}")
print()
print("Note: FeasibilityFirst penalizes infeasible solutions heavily.")
print("      Epsilon with ε=0.15 treats solution 4 (CV=0.1) as feasible.")

## 3. Constrained Engineering Problem: Welded Beam

The welded beam design problem has:
- **6 variables**: weld thickness, length, flange thickness, width, material choice, stiffener count
- **2 objectives**: minimize cost, minimize deflection
- **4 constraints**: shear stress, normal stress, deflection limit, geometric coupling

In [ ]:
# Create welded beam problem
welded_beam = WeldedBeamDesignProblem()

print(f"Problem: Welded Beam Design")
print(f"  Variables: {welded_beam.n_var}")
print(f"  Objectives: {welded_beam.n_obj}")
print(f"  Encoding: {welded_beam.encoding}")
print()
print("Variables:")
print("  x0: weld thickness h ∈ [0.125, 5]")
print("  x1: length l ∈ [0.1, 10]")
print("  x2: flange thickness t ∈ [0.1, 10]")
print("  x3: width b ∈ [0.125, 5]")
print("  x4: material choice ∈ {0, 1, 2}")
print("  x5: stiffener count ∈ {2, ..., 6}")
print()
print("Constraints (G ≤ 0 = feasible):")
print("  g1: shear stress ≤ limit")
print("  g2: normal stress ≤ limit")
print("  g3: deflection ≤ 0.25")
print("  g4: h ≤ b (geometric)")

In [ ]:
# Configure algorithm for constrained problem
constrained_config = (
    NSGAIIConfig()
    .pop_size(100)
    .crossover("sbx", prob=0.9, eta=15.0)
    .mutation("pm", prob="1/n", eta=20.0)
    .engine("numpy")
    .fixed()
)

# Run optimization
welded_result = optimize(OptimizeConfig(
    problem=welded_beam,
    algorithm="nsgaii",
    algorithm_config=constrained_config,
    termination=("n_eval", 10000),
    seed=42,
))

print(f"Found {len(welded_result.F)} solutions")

In [ ]:
# Analyze feasibility of final population
if welded_result.G is not None:
    cinfo = compute_constraint_info(welded_result.G)
    n_feasible = cinfo.feasible_mask.sum()
    n_total = len(cinfo.feasible_mask)
    
    print(f"Feasibility analysis:")
    print(f"  Feasible solutions: {n_feasible} / {n_total} ({100*n_feasible/n_total:.1f}%)")
    print(f"  Average CV (all): {cinfo.cv.mean():.4f}")
    if not cinfo.feasible_mask.all():
        print(f"  Average CV (infeasible only): {cinfo.cv[~cinfo.feasible_mask].mean():.4f}")
else:
    print("No constraint data returned.")
    cinfo = None

In [ ]:
# Visualize: color by feasibility
fig, ax = plt.subplots(figsize=(8, 6))

if cinfo is not None:
    feasible = cinfo.feasible_mask
    
    # Plot infeasible (if any)
    if (~feasible).any():
        ax.scatter(welded_result.F[~feasible, 0], welded_result.F[~feasible, 1],
                   s=30, alpha=0.5, c='red', label=f'Infeasible ({(~feasible).sum()})', marker='x')
    
    # Plot feasible
    if feasible.any():
        ax.scatter(welded_result.F[feasible, 0], welded_result.F[feasible, 1],
                   s=40, alpha=0.7, c='green', label=f'Feasible ({feasible.sum()})')
else:
    ax.scatter(welded_result.F[:, 0], welded_result.F[:, 1], s=40, alpha=0.7, c='blue')

ax.set_xlabel("Cost")
ax.set_ylabel("Deflection")
ax.set_title("Welded Beam: Cost vs Deflection (Constrained)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. CEC2009 Constrained Test Function (CF1)

The CEC2009 CF suite provides challenging constrained multi-objective problems.

In [ ]:
# Create CF1 problem
cf1 = CEC2009_CF1(n_var=10)

print(f"Problem: CEC2009 CF1")
print(f"  Variables: {cf1.n_var}")
print(f"  Objectives: {cf1.n_obj}")
print(f"  Bounds: [{cf1.xl[0]:.1f}, {cf1.xu[0]:.1f}]")
print()
print("The CF1 constraint creates infeasible bands across the Pareto front,")
print("making parts of the objective space unreachable.")

In [ ]:
# Run on CF1
cf1_config = (
    NSGAIIConfig()
    .pop_size(100)
    .crossover("sbx", prob=0.9, eta=20.0)
    .mutation("pm", prob="1/n", eta=20.0)
    .engine("numpy")
    .fixed()
)

cf1_result = optimize(OptimizeConfig(
    problem=cf1,
    algorithm="nsgaii",
    algorithm_config=cf1_config,
    termination=("n_eval", 15000),
    seed=42,
))

print(f"Found {len(cf1_result.F)} solutions")

In [ ]:
# Analyze CF1 feasibility
if cf1_result.G is not None:
    cf1_cinfo = compute_constraint_info(cf1_result.G)
    n_feas = cf1_cinfo.feasible_mask.sum()
    print(f"Feasible: {n_feas} / {len(cf1_cinfo.feasible_mask)}")
else:
    cf1_cinfo = None
    print("No constraint data.")

In [ ]:
# Visualize CF1 front with feasibility
fig, ax = plt.subplots(figsize=(8, 6))

if cf1_cinfo is not None:
    feasible = cf1_cinfo.feasible_mask
    
    if (~feasible).any():
        ax.scatter(cf1_result.F[~feasible, 0], cf1_result.F[~feasible, 1],
                   s=30, alpha=0.4, c='red', label='Infeasible', marker='x')
    if feasible.any():
        ax.scatter(cf1_result.F[feasible, 0], cf1_result.F[feasible, 1],
                   s=40, alpha=0.7, c='green', label='Feasible')
else:
    ax.scatter(cf1_result.F[:, 0], cf1_result.F[:, 1], s=40, alpha=0.7)

# Plot true Pareto front reference (f1 + f2 = 1 for unconstrained)
f1_ref = np.linspace(0, 1, 100)
f2_ref = 1 - f1_ref
ax.plot(f1_ref, f2_ref, 'k--', alpha=0.3, label='Unconstrained PF')

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("CEC2009 CF1: Constrained Pareto Front")
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.05, 1.1)
ax.set_ylim(-0.05, 1.1)
plt.tight_layout()
plt.show()

## 5. Constraint DSL (Symbolic Constraints)

VAMOS provides a DSL for building constraints symbolically, useful for custom problems.

In [ ]:
from vamos.foundation.constraints.dsl import constraint_model, build_constraint_evaluator

# Define constraints: x0 + x1 <= 1, x0 >= 0.2, x1 >= 0.1
with constraint_model(n_vars=2) as cm:
    x0, x1 = cm.vars("x0", "x1")
    cm.add(x0 + x1 <= 1.0)      # Sum constraint
    cm.add(x0 >= 0.2)           # Lower bound on x0
    cm.add(x1 >= 0.1)           # Lower bound on x1

# Build evaluator
eval_constraints = build_constraint_evaluator(cm)

# Test on sample points
X_test = np.array([
    [0.3, 0.4],  # Feasible: sum=0.7 <= 1, x0=0.3 >= 0.2, x1=0.4 >= 0.1
    [0.1, 0.5],  # Infeasible: x0=0.1 < 0.2
    [0.6, 0.6],  # Infeasible: sum=1.2 > 1
    [0.5, 0.05], # Infeasible: x1=0.05 < 0.1
])

G_test = eval_constraints(X_test)
test_info = compute_constraint_info(G_test)

print("DSL constraint evaluation:")
print(f"  G shape: {G_test.shape}")
print(f"  Constraints: x0+x1<=1, x0>=0.2, x1>=0.1")
print()
for i, (x, g, feas) in enumerate(zip(X_test, G_test, test_info.feasible_mask)):
    status = "✓ Feasible" if feas else "✗ Infeasible"
    print(f"  Point {i}: x={x} → G={g} → {status}")

## 6. Visualizing Constraint Violations

In [ ]:
# Analyze per-constraint violations for welded beam
if welded_result.G is not None:
    G = welded_result.G
    constraint_names = ["Shear stress", "Normal stress", "Deflection", "h ≤ b"]
    
    fig, axes = plt.subplots(2, 2, figsize=(10, 8))
    axes = axes.flatten()
    
    for i, (ax, name) in enumerate(zip(axes, constraint_names)):
        violations = G[:, i]
        satisfied = violations <= 0
        
        ax.hist(violations, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
        ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Feasibility boundary')
        ax.set_xlabel(f"g{i+1} value")
        ax.set_ylabel("Count")
        ax.set_title(f"{name}: {satisfied.sum()}/{len(satisfied)} satisfied")
        ax.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No constraint data available.")

## 7. Best Feasible Solutions

In [ ]:
# Extract best feasible solutions from welded beam
if cinfo is not None and cinfo.feasible_mask.any():
    feasible_idx = np.where(cinfo.feasible_mask)[0]
    F_feasible = welded_result.F[feasible_idx]
    X_feasible = welded_result.X[feasible_idx]
    
    # Find extremes
    best_cost_idx = feasible_idx[np.argmin(F_feasible[:, 0])]
    best_deflection_idx = feasible_idx[np.argmin(F_feasible[:, 1])]
    
    print("Best feasible solutions:")
    print("="*60)
    
    for name, idx in [("Minimum Cost", best_cost_idx), ("Minimum Deflection", best_deflection_idx)]:
        x = welded_result.X[idx]
        f = welded_result.F[idx]
        print(f"\n{name}:")
        print(f"  Cost: {f[0]:.2f}, Deflection: {f[1]:.4f}")
        print(f"  h={x[0]:.3f}, l={x[1]:.3f}, t={x[2]:.3f}, b={x[3]:.3f}")
        print(f"  Material={int(round(x[4]))}, Stiffeners={int(round(x[5]))}")
else:
    print("No feasible solutions found or no constraint data.")

## Summary

**Constraint Convention:**
- `G ≤ 0` = constraint satisfied (feasible)
- `G > 0` = constraint violated (infeasible, violation = G value)

**Handling Strategies:**
| Strategy | Best For |
|----------|----------|
| `FeasibilityFirstStrategy` | Strict feasibility requirements |
| `PenaltyCVStrategy` | Balancing feasibility with objective quality |
| `EpsilonConstraintStrategy` | Soft constraint boundaries |

**Tips:**
- Use `compute_constraint_info(G)` to analyze violations
- Constrained problems often need larger populations and more evaluations
- Visualize feasible vs infeasible to understand constraint difficulty
- The constraint DSL helps build custom constraints symbolically